## Imports and declerations

In [1]:
import pickle
import os
import string

import re
import math
import pprint
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import copy
from copy import deepcopy
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
InteractiveShell.ast_node_interactivity = "all"




## Data Structure:

In [2]:
class PostingList(object):
    def __init__(self):
        self.total_count = 0
        self.token = ''
        self.occurrance = {
#             'doc_id':0 = 'positions' : [],
#             
        }
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)
         
    def __repr__(self):
        
        return f'total_cnt : {self.total_count} docs : [{self.occurrance.keys()}]'
    def __len__(self):
        return len(self.occurrance)
    def addOccurrance(self, doc_id, position):
        self.total_count += 1
#         print(position)
        if doc_id not in self.occurrance.keys():
            self.occurrance[doc_id] = []
        self.occurrance[doc_id].append(position)
#         self.occurrance[doc_id]['position'].append(pos)
    
class InvertedIndex(object):
    
    def __init__(self):
        self.index = {}
        self.docs = {}

    def get_term_postings(self, term):
        if term in self.index.keys():
            return self.index[term]
        else:
            raise ValueError(f'{term} is not present in Index. Plese Try Again')
            return PostingList()
    
    def __len__(self):
        return len(self.index.keys())
    
# vector_hash = {'term1':cnt1,,,,} // v_hash
# vector_arr = [cnt0,cnt1,cnt2 ...] // v_arr

class VectorSpaceModel(object):
    def __init__(self):
        self.vocab = {} # type = dict -> keys = terms in docs, values = count of terms. e.g {'global': 25, ...}
        self.expanded_vocab = {} # type =  dict -> keys =  synonyms of terms in docs, values = count of term related to synonyms in docs e.g
        self.group_vocab = {} # type = dict ->  groups of synonyms in docs,  values = count of all synonyms in docs. e.g {('trump','trump_card','cornet','horn'): 310}
        
        self.vocab_idf = {} # calculated idf of terms 
        self.group_vocab_idf = {} # calculated idf of term groups
        
        self.docs = {}
        self.occurrance = {}
        self.cdocs = {}
        self.group_cdocs = {}
        
        self.index = {}
        self.group_index = {}
        """
        Type = dict -> keys = terms in docs, values = synonyms groups of term.
        e.g 
         'close_to': ('roughly',
                      'around',
                      'or_so',
                      'some',
                      'approximately',
                      'more_or_less',
                      'just_about',
                      'about',
                      'close_to'),
        """
        self.term_group = {} 
        """
        syn_group refers to synonyms group of a term.
        syn1 refers to 1st synonym of synonyms group of a term. 
        Type : dict -> keys = docId, values = dict with syn_group as key and count as value
        e.g
            {
                "docId":
                {
                    (syn1, syn2, syn3, ... ) : count_occurance_of_any_syn_in_group
                }
            }
        
        """
        self.group_docs = {}
        
        
    def create_doc(self, docId):
        self.docs[docId] = dict.fromkeys(self.vocab, 0)
        self.group_docs[docId] = dict.fromkeys(self.group_vocab, 0)
        
    def get_synonyms(self, term):
        synonyms = []
        for syn in wordnet.synsets(term):
            for l in syn.lemmas():
                synonyms.append(l.name()) 
        if term not in synonyms:
            synonyms.append(term)
        return tuple(set(synonyms))
    
    def add_term(self, term, docId):
        
        if term in self.expanded_vocab.keys():
            synonyms = self.term_group[term]
            self.group_vocab[synonyms] += 1
            for related_word in synonyms:
                related_word= related_word.lower()
                self.expanded_vocab[related_word] += 1
            if synonyms not in self.group_docs[docId].keys():
                self.group_docs[docId][synonyms] = 1
            else:
                self.group_docs[docId][synonyms] += 1
            self.group_index[synonyms].add(docId)
        
        else:
            if term == 'muslim':
                print('HOW THE HELL')
            synonyms = self.get_synonyms(term)
            if len(synonyms) > 0:
                if synonyms not in self.group_vocab.keys():
                    self.group_vocab[synonyms] = 1
                else:
                    self.group_vocab[synonyms] += 1

                for related_word in synonyms:
                    related_word= related_word.lower()
                    if related_word not in self.expanded_vocab.keys():
                        self.expanded_vocab[related_word] = 1
                    else:
                        self.expanded_vocab[related_word] += 1

                    if related_word not in self.term_group.keys():
                        self.term_group[related_word] = synonyms

                if synonyms not in self.group_docs[docId].keys():
                    self.group_docs[docId][synonyms] = 1
                else:
                    self.group_docs[docId][synonyms] += 1

                if synonyms in self.group_index.keys():
                    self.group_index[synonyms].add(docId)
                else:
                    self.group_index[synonyms] = set()
                    self.group_index[synonyms].add(docId)
                
        
        if term not in self.vocab.keys():
            self.vocab[term] = 1
            for Id, docList in self.docs.items():
                self.docs[Id][term] = 0
        else:
            self.vocab[term]+=1
        
        if term in self.docs[docId].keys():
            self.docs[docId][term] += 1
        else:
            self.docs[docId][term] = 1
        
        
        
        
        if term in self.index.keys():
            self.index[term].add(docId)
        else:
            self.index[term] = set()
            self.index[term].add(docId)
    
#         if docId not in self.occurrance.keys():
#             self.occurrance[docId] = []
#         self.occurrance[docId].append(position)

    def get_query_group_vector(self, query_terms):
        query_group_vector_hash = dict.fromkeys(self.group_vocab, 0)
#         print(query_vector_hash)
        
        for term in query_terms:
            term = lem.lemmatize(term.lower())
            if term in self.term_group.keys():
                synonyms = self.term_group[term]
                if synonyms in query_group_vector_hash.keys():
    #                 print('Getting Synonyms')
    #                 print(term)
    #                 print(synonyms)
                    query_group_vector_hash[synonyms] += 1

        
            groups_in_query = len(query_terms)
            tf = dict.fromkeys(self.group_vocab_idf, 0)
            idf = dict.fromkeys(self.group_vocab_idf, 0)
            tf_idf = dict.fromkeys(self.group_vocab_idf, 0)
            
            for syn_group,syn_group_cnt in query_group_vector_hash.items():
                if syn_group_cnt <= 0:
                    continue
#                 print(syn_group)
#                 print(syn_group_cnt)
#                 if not isinstance(syn_group_cnt, int):
#                     print(syn_group)
#                     print(syn_group_cnt)
#                 tf[term] = term_cnt / words_in_query
#                 idf[term] = len(self.docs.keys()) / (self.vocab[term])
                
                tf[syn_group] = syn_group_cnt / groups_in_query
                tf_idf[syn_group] = tf[syn_group] * self.group_vocab_idf[syn_group]
        
            new_query_vector_hash = {
                'tf': tf,
                'idf':self.group_vocab_idf,
                'tf_idf':tf_idf
            }
    
    
        return new_query_vector_hash




    def get_query_vector(self, query_terms):
        query_vector_hash = dict.fromkeys(self.vocab, 0)
#         print(query_vector_hash)
        for term in query_terms:
            if term in query_vector_hash.keys():
                query_vector_hash[term] += 1
#             else:
#                 query_vector_hash[term] = 1
        
            words_in_query = len(query_terms)
            tf = {}
            idf = {}
            tf_idf = {}
            
            for term,term_cnt in query_vector_hash.items():
#                 print(term)
#                 print(term_cnt)
                if not isinstance(term_cnt, int):
                    print(term)
                    print(term_cnt)
#                 tf[term] = term_cnt / words_in_query
#                 idf[term] = len(self.docs.keys()) / (self.vocab[term])
                
                tf[term] = term_cnt/ words_in_query
                idf[term] = self.vocab_idf[term]
                tf_idf[term] = tf[term] * idf[term]
        
            new_query_vector_hash = {
                'tf': tf,
                'idf':idf,
                'tf_idf':tf_idf
            }
    
    
        return new_query_vector_hash
    
    def dot_product(self, v_hash_1, v_hash_2):
#         print(len(v_hash_1))
#         print(len(v_hash_2))
#         print('vhash')
#         print(v_hash_2)
        return sum([v_hash_2[x]*y for x,y in v_hash_1.items()])
        
    def get_magnitude(self, v_hash):
        mag = sum([x**2 for x in v_hash.values()]) ** 0.5 
        if mag == 0:
            print('Boy we got zero')
            print([x for x in v_hash.values() if x > 0])
            return 1
        return mag
    
    
    def get_cosine_sim(self, v_hash_x, v_hash_y):
        
        return (self.dot_product(v_hash_x,v_hash_y)) / (self.get_magnitude(v_hash_x) * self.get_magnitude(v_hash_y)) 
            
    def get_ranking(self, query_vector_hash):
        ranked_docs = []
        for docId,doc_vector_hash in self.cdocs.items():
            cosine_sim = self.get_cosine_sim(doc_vector_hash['tf_idf'], query_vector_hash['tf_idf'])
            ranked_docs.append((docId, cosine_sim))
        ranked_docs = sorted(ranked_docs, key=lambda x:x[1])
        return ranked_docs
    
    def get_group_ranking(self, query_vector_hash):
        ranked_docs = []
        for docId,doc_vector_hash in self.group_cdocs.items():
            cosine_sim = self.get_cosine_sim(doc_vector_hash['tf_idf'], query_vector_hash['tf_idf'])
            ranked_docs.append((docId, cosine_sim))
        ranked_docs = sorted(ranked_docs, key=lambda x:x[1])
        return ranked_docs
    
    def calculate_tf_idf(self):
        cdocs = {}
        
        self.vocab_idf = dict.fromkeys(self.vocab, 0)
        self.group_vocab_idf = dict.fromkeys(self.group_vocab, 0)
        
        
        for syn_group, group_cnt in self.group_vocab.items():
            self.group_vocab_idf[syn_group] = math.log10((len(self.docs.keys()) / (len(self.group_index[syn_group])))+1)
        
        for term, term_cnt in self.vocab.items():
            self.vocab_idf[term] = math.log10((len(self.docs.keys()) / (len(self.index[term]))) + 1 )
            
        for docId,doc in self.docs.items():
#             print(doc.values())
            words_in_d = sum(doc.values())
            tf = {}
        
            tf_idf = {}
            for term,term_cnt in doc.items():
#                 tf[term] = term_cnt / words_in_d
#                 idf[term] = len(self.docs.keys()) / (self.vocab[term])
                tf[term] = term_cnt / words_in_d
                tf_idf[term] = tf[term] * self.vocab_idf[term]
            cdocs[docId] = {
                'tf': tf,
                'tf_idf':tf_idf,
                'total_words' : words_in_d
            }
        group_cdocs = {}
        for docId,doc in self.group_docs.items():
#             print(doc.values())
            groups_in_d = sum(doc.values())
            tf = {}
        
            tf_idf = {}
            for syn_group,syn_group_cnt in doc.items():
#                 tf[term] = term_cnt / words_in_d
#                 idf[term] = len(self.docs.keys()) / (self.vocab[term])
                tf[syn_group] = syn_group_cnt / groups_in_d
                tf_idf[syn_group] = tf[syn_group] * self.group_vocab_idf[syn_group]
            group_cdocs[docId] = {
                'tf': tf,
                'tf_idf':tf_idf,
                'total_words' : groups_in_d
            }
        self.cdocs = cdocs
        self.group_cdocs = group_cdocs
        return self.cdocs
            
        

In [3]:
test_space = VectorSpaceModel()

d1= "Music is a universal language"
d2= "Music is a miracle"
d3= "Music is a universal feature of the human experience"
test_space.create_doc(1)
for word in d1.split(' '):
    test_space.add_term(word, 1)
test_space.create_doc(2)
for word in d2.split(' '):
    test_space.add_term(word, 2)
test_space.create_doc(3)
for word in d3.split(' '):
    test_space.add_term(word, 3)
# test_space.term_group
test_space.expanded_vocab['music']
test_space.expanded_vocab['euphony']
# test_space.calculate_tf_idf()


6

3

## Helper Functions:

In [4]:
# Remove Punctuation
def remove_punctuation(word):
    return word.translate(word.maketrans('','',string.punctuation))

# Clean Query Term
def clean_word(word):
    # Case Folding
    word = word.lower()
     # Filter non-ASCII characters
    word = ''.join(filter(lambda x: x in printable, word))
#     print(word)
    # Remove Punctuations
    if word != '(' and word != ')':
        word = remove_punctuation(word)
#     print(word)
    if re.match('\d+[A-Za-z]+',word):
        word = re.split('\d+',word)[1]
    if re.match('[A-Za-z]+\d+',word):
        word = re.split('\d+',word)[0]
#     print(word)
    word = ps.stem(word)
#     print(word)
    return word

## Term - Document Indexing:

In [5]:
vocab = set()
doc_contents = []
inverted_index = InvertedIndex()
vector_space = VectorSpaceModel()
printable = set(string.printable) 
raw_data = []
# Printable characters are
# 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
# !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c


        
        
lem = WordNetLemmatizer() 

stop_words = set()
with open('Stopword-List.txt', 'r') as stop_word_file:
    lines = stop_word_file.readlines()
    for line in lines:
        stop_words.add(line.split('\n')[0])
    stop_words.remove('')

    
print('Loading : ', end='')
for file_number in range(0, 56):
    vector_space.create_doc(file_number)
    with open(f'data/Trump Speechs/speech_{file_number}.txt', 'r') as file1:
        lines = file1.readlines()
#         print(f'File Number : speech_{file_number}.txt' )
#         print(lines[0])
        position = {'doc':file_number,'row':0, 'col':0, 'token_no':0}
        
#         {
#             'total_count' : 0,
#             'postings' : {
#                 'count':0,
#                 'doc_id':0,
#                 'positions':[]
#             }
#         }
        
        for line_no,line in enumerate(lines):
            # Skip Heading Line
            if line_no == 0:
                continue
            doc_set = set()
            raw_data.append(line)
            # split words at . , whitespace ? ! : ;
            position['row'] = line_no 
            position['col'] = 0
            symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
            for i in symbols:
                line = line.replace(i, ' ')
            
            for word in re.split('[.\s,?!:;-]', line):
                position['col'] += len(word) + 1
                position['token_no'] += 1
                # Case Folding
                word = word.lower()
                
                # Filter non-ASCII characters
                word = ''.join(filter(lambda x: x in printable, word))
                
                # Remove Punctuations
                word = remove_punctuation(word)
                
                if re.match('\d+[A-Za-z]+',word):
                    word = re.split('\d+',word)[1]
                if re.match('[A-Za-z]+\d+',word):
                    word = re.split('\d+',word)[0]
                
                if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
                    continue
                if word in stop_words:
                    continue
                
                
                
                word = lem.lemmatize(word)
    
                    
                vocab.add(word)
                
                doc_set.add(word)
                
                vector_space.add_term(word, file_number)
                
                
                if word in inverted_index.index.keys():
                    
                    inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position)) 
                else:
                    plist = PostingList()
                    inverted_index.index[word] = plist
                    inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position))
                    
        inverted_index.docs[file_number] = doc_set
        doc_contents.append(doc_set)
        print('*', end='')
doc_term_tf_idf = vector_space.calculate_tf_idf()

print('Done')

Loading : ********************************************************Done


In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(raw_data)
print('Total Vocabulary Size ')
print(len(vectorizer.get_feature_names()))
print(X.toarray())

y = vectorizer.transform(['muslims'])
print(y)
print(y.toarray())
print(y.todense())

tfidf = TfidfTransformer()
tfidf.fit(X)
tf_idf_matrix = tfidf.transform(X)
query_tf_idf = tfidf.transform(y)

print(tf_idf_matrix.todense()[0])
tf_idf_matrix.shape
cosine_similarity(query_tf_idf, tf_idf_matrix)
# print([y for x,y in sorted(vector_space.cdocs[0]['tf_idf'].items(), key = lambda x: x[0]) ])

Total Vocabulary Size 
7014
[[1 1 0 ... 6 0 0]
 [0 2 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 2 0 ... 0 0 0]
 [0 3 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
  (0, 4136)	1
[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]


TfidfTransformer(norm='l1', smooth_idf=True, sublinear_tf=False, use_idf=True)

[[0.00043717 0.00011168 0.         ... 0.00196056 0.         0.        ]]


(56, 7014)

array([[0.        , 0.        , 0.        , 0.01269782, 0.02366651,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [11]:
print('Total Vocabulary Size ')
print(len(inverted_index.index.keys()))
print('Total Number of Documents ')
print(len(inverted_index.docs))
# print('Occurances of query: hammer')
# inverted_index.index[lem.lemmatize('interest')].occurrance
# import nltk
# nltk.download('wordnet')
# vector_space.index[lem.lemmatize('hillary')]
lem.lemmatize('clinton')
# vector_space.cdocs[0]

Total Vocabulary Size 
6249
Total Number of Documents 
56


'clinton'

In [12]:
query = "Hillary clinton"
query_terms = [lem.lemmatize(x) for x in query.split(' ')]
print(query_terms)

query_vector = vector_space.get_query_vector(query_terms)

# cs = vector_space.get_cosine_sim(vector_space.docs[1], query_vector)
# print([(x,y) for x,y in query_vector.items() if y > 0])
# ranked = vector_space.get_ranking(vector_space.get_query_vector(query_terms))
# ranked.reverse()
# ranked = set([x for x,y in ranked if y > 0.0005])
# print(ranked)

vector_space.cdocs[41]['tf'][lem.lemmatize('global')]

['Hillary', 'clinton']


0.0

In [13]:
test = [(None, None) for x in range(0, 17)] 
test[0] =("massive inflow of refugees",
{'32', '50', '49', '47', '46', '29', '48', '54', '41', '40', '30', '39', '12', '52', '37', '44', '31', '38', '20'}
         )
test[1] =('pakistan afghanistan',
{'3', '22', '16', '17', '4', '1'}
         )
test[2] =('Hillary Clinton',
{'4', '12', '36', '42', '33', '20', '29', '35', '16', '22', '47', '34', '14', '21', '45', '46', '41', '11', '3', '51', '53', '40', '37', '17', '48', '39', '5', '28', '10', '18', '54', '44', '25', '43', '49', '24', '30', '32', '8', '50', '9', '26', '7', '19', '52', '31', '27', '6', '2', '38', '1', '55'}
         )
test[3] =('personnel policies',

{'5', '18', '11', '25', '10', '22', '29', '27', '17'}
         )

test[4] =('united plane',
{'38', '1', '25', '41', '48', '29', '3', '17', '49', '20', '4', '52', '13', '39', '46', '22', '40', '36', '28', '12', '45', '26', '2', '51', '50', '19', '24', '5', '47', '31', '35', '21', '37', '55', '9', '33', '44', '54', '34', '7', '32', '43', '30', '16', '27', '18', '11', '10', '8'}
         )

test[5] =('develop solutions',
{'38', '23', '2', '17', '32', '18', '51', '16', '39', '35', '21', '27', '33', '20', '24', '5', '3', '9', '40', '52', '30'}
         )

test[6] =('developments praised',
{'36', '44', '8'}
         )
test[7] =("muslims",
{'4', '3'}
         )
test[8] =('American Energy Revolution',

{'13', '28', '35', '33', '51', '27', '12', '36', '34', '14', '21', '29', '31', '30', '25', '32', '26', '24', '11', '54', '49', '16', '46', '48', '10', '18', '4', '50', '42', '5', '23', '47', '20', '43', '37', '39', '41', '45', '2', '22', '3', '40', '7', '17', '52', '44', '53', '19', '9', '55', '1', '15', '8'}
         )
test[9] =('Future of new America',
{'51', '14', '35', '26', '13', '50', '33', '4', '12', '31', '20', '32', '29', '34', '41', '27', '46', '30', '40', '25', '17', '42', '16', '24', '54', '2', '43', '21', '7', '18', '52', '49', '5', '47', '45', '10', '36', '38', '44', '11', '3', '39', '22', '9', '48', '37', '23', '15', '1', '19', '55', '6', '8'}
         )
test[10] =('Hillary clinton is the worst looser',
{'4', '12', '36', '42', '33', '20', '29', '35', '16', '22', '47', '34', '14', '21', '45', '46', '41', '11', '3', '51', '53', '40', '37', '17', '48', '39', '5', '28', '10', '18', '54', '44', '25', '43', '49', '24', '30', '32', '8', '50', '9', '26', '7', '19', '52', '31', '27', '6', '2', '38', '1', '55'}
          )
test[11] =('no patience for injustice',
{'11', '22', '16', '7', '15'}
          )
test[12] =('Global interests',
{'10', '16', '27', '42', '21', '11', '25', '22', '47', '41', '48', '54', '18', '40', '24', '39', '8'}
          )
test[13] =('pakistan afghanistan aid',
{'3', '22', '42', '29', '41', '16', '40', '39', '17', '4', '1'}
          )
test[14] =('biggest plane wanted hour',
{'52', '1', '19'}
          )
test[15] =('near architect box',
{'54', '23', '18', '24', '39', '43', '51', '4', '9', '17', '11'}
          )
test[16] =('peaceful change',
{'14', '10', '4', '30', '3', '11', '20', '16', '50', '52', '48', '17', '7', '2', '22', '29', '13', '53', '41', '46', '32', '21', '39', '25', '33', '54', '42', '5', '36', '31', '43', '51', '49', '23', '47', '35', '37', '12', '45', '8'}
         )

cumulative_precision_normal = 0
cumulative_recall_normal = 0 
cumulative_precision_sciket = 0
cumulative_recall_sciket = 0 
cumulative_precision_group = 0
cumulative_recall_group = 0 

#         tp, fp, tn
sciket_results = []
cnt = 0
for t in test:
    cnt+=1
    normal_cnt = [0, 0 , 0]
    sciket_cnt = [0, 0, 0]
    group_cnt = [0, 0, 0]
    
    # sciket model
    y = vectorizer.transform([t[0]])
    query_tf_idf = tfidf.transform(y)
    sciket_cosines = cosine_similarity(query_tf_idf, tf_idf_matrix)[0]
    # my custom model
    query = t[0]
    query_terms = [lem.lemmatize(x.lower()) for x in query.split(' ')]
    print('\t' + str(query_terms))

    query_vector = vector_space.get_query_vector(query_terms)
    query_group_vector = vector_space.get_query_group_vector(query_terms)
    # cs = vector_space.get_cosine_sim(vector_space.docs[1], query_vector)
    # print([(x,y) for x,y in query_vector.items() if y > 0])
    ranked = vector_space.get_ranking(vector_space.get_query_vector(query_terms))
    ranked.reverse()
    ranked = set([str(x) for x,y in ranked if y > 0.0005])
#     print(ranked)
    print('Normal results')
    print(f'results length :  {len(ranked)}')
    print('Result Differnece : ')
    print(t[1].difference(ranked))
    print('Symmetric Difference : ')
    print(t[1].symmetric_difference(ranked))
    print()
    normal_cnt = [len(t[1]), len(ranked.difference(t[1])), len(t[1].difference(ranked))]
    normal_p = normal_cnt[0] / (normal_cnt[0] + normal_cnt[1])
    normal_r = normal_cnt[0] / (normal_cnt[0] + normal_cnt[2])
    print(f'precision : {normal_p}')
    print(f'recall : {normal_r}')
    cumulative_precision_normal += normal_p
    cumulative_recall_normal += normal_r
    
#     Sciket results
    print('Sciket results')
    sciket_ranked = [str(index) for index, cosine in enumerate(sciket_cosines) if cosine > 0.0005]
    print(f'results length :  {len(sciket_ranked)}')
    print('Result Differnece : ')
    print(t[1].difference(set(sciket_ranked)))
    print('Symmetric Difference : ')
    print(t[1].symmetric_difference(set(sciket_ranked)))
    print()
    sciket_cnt = [len(t[1]), len(set(sciket_ranked).difference(t[1])), len(t[1].difference(set(sciket_ranked)))]
    sciket_p = sciket_cnt[0] / (sciket_cnt[0] + sciket_cnt[1])
    sciket_r = sciket_cnt[0] / (sciket_cnt[0] + sciket_cnt[2])
    print()
    print(f'precision : {sciket_p}')
    print(f'recall : {sciket_r}')
    cumulative_precision_sciket += sciket_p
    cumulative_recall_sciket += sciket_r
# Group Results 
    group_ranked = vector_space.get_group_ranking(query_group_vector)
    group_ranked.reverse()
    group_ranked = set([str(x) for x,y in group_ranked if y > 0.0005])
    
    print('Group results')
    print(f'results length :  {len(group_ranked)}')
    print('Result Differnece : ')
    print(t[1].difference(set(group_ranked)))
    print('Symmetric Difference : ')
    print(t[1].symmetric_difference(set(group_ranked)))
    print()
    group_cnt = [len(t[1]), len(group_ranked.difference(t[1])), len(t[1].difference(group_ranked))]
    group_p = group_cnt[0] / (group_cnt[0] + group_cnt[1])
    group_r = group_cnt[0] / (group_cnt[0] + group_cnt[2])
    print(f'precision : {group_p}')
    print(f'recall : {group_r}')
    cumulative_precision_group += group_p
    cumulative_recall_group += group_r
    
f1_normal = (2 * (cumulative_precision_normal / cnt) * (cumulative_recall_normal / cnt) ) / (cumulative_recall_normal+cumulative_precision_normal)
f1_sciket = (2 * (cumulative_precision_sciket / cnt) * (cumulative_recall_sciket / cnt) ) / (cumulative_recall_sciket+cumulative_precision_sciket)
f1_group = (2 * (cumulative_precision_group / cnt) * (cumulative_recall_group / cnt) ) / (cumulative_recall_group+cumulative_precision_group)

print(f'normal f1 : {f1_normal}')
print(f'sciket f1 : {f1_sciket}')
print(f'group f1 : {f1_group}')

	['massive', 'inflow', 'of', 'refugee']
Normal results
results length :  56
Result Differnece : 
set()
Symmetric Difference : 
{'33', '27', '16', '23', '7', '22', '6', '55', '25', '18', '4', '28', '35', '0', '5', '42', '19', '3', '24', '10', '1', '15', '21', '43', '9', '45', '2', '17', '13', '8', '36', '14', '34', '11', '26', '53', '51'}

precision : 0.3392857142857143
recall : 1.0
Sciket results
results length :  56
Result Differnece : 
set()
Symmetric Difference : 
{'33', '27', '16', '23', '7', '6', '22', '55', '25', '18', '4', '28', '35', '0', '5', '19', '42', '3', '10', '24', '51', '1', '15', '21', '43', '9', '45', '2', '17', '13', '36', '14', '34', '11', '26', '53', '8'}


precision : 0.3392857142857143
recall : 1.0
Group results
results length :  56
Result Differnece : 
set()
Symmetric Difference : 
{'33', '27', '16', '23', '7', '22', '6', '55', '25', '18', '4', '28', '35', '0', '5', '42', '19', '3', '24', '10', '1', '15', '21', '43', '9', '45', '2', '17', '13', '8', '36', '14', 

Group results
results length :  56
Result Differnece : 
set()
Symmetric Difference : 
{'33', '23', '7', '55', '6', '46', '4', '32', '28', '37', '35', '31', '38', '12', '49', '0', '5', '19', '3', '20', '52', '1', '44', '15', '50', '43', '29', '9', '45', '30', '2', '17', '13', '36', '34', '14', '26', '53', '51'}

precision : 0.30357142857142855
recall : 1.0
	['pakistan', 'afghanistan', 'aid']
Normal results
results length :  14
Result Differnece : 
set()
Symmetric Difference : 
{'37', '9', '18'}

precision : 0.7857142857142857
recall : 1.0
Sciket results
results length :  14
Result Differnece : 
set()
Symmetric Difference : 
{'9', '18', '37'}


precision : 0.7857142857142857
recall : 1.0
Group results
results length :  51
Result Differnece : 
set()
Symmetric Difference : 
{'33', '27', '23', '55', '6', '25', '48', '18', '46', '28', '32', '37', '35', '31', '38', '49', '0', '5', '19', '10', '52', '24', '15', '44', '50', '21', '43', '9', '45', '30', '2', '13', '8', '54', '36', '14', '34', '4

In [522]:
# To save Inverted Index in File
with open('pickled/inverted_index.p', 'wb') as index_file:
    pickle.dump(inverted_index, index_file)

In [45]:
# To Load Inverted Index from File
with open('pickled/inverted_index.p', 'rb') as index_file:
    inverted_index = pickle.load(index_file)

In [9]:
def intersect_posting(p1, p2):
    if len(p1) == 0 or len(p2) == 0:
        return PostingList()

    if isinstance(p1, set) and isinstance(p2, set):
        return p1.intersection(p2)
    elif isinstance(p1, set):
        return p1.intersection(p2.occurrance.keys())
    elif isinstance(p2, set):
        return p2.intersection(p1.occurrance.keys())
    pn = PostingList()
    # pn.token = f'{p1.token} & {p2.token}'
    for pn_keys in (p1.occurrance.keys() & p2.occurrance.keys()) :
        pn.addOccurrance(pn_keys, p1.occurrance[pn_keys])
        pn.addOccurrance(pn_keys, p2.occurrance[pn_keys])
    return pn


def union_posting(p1, p2):
    if len(p1) == 0:
        return p2
    elif len(p2) == 0:
        return p1
    if isinstance(p1, set) and isinstance(p2, set):
        return p1.union(p2)
    elif isinstance(p1, set):
        return p1.union(p2.occurrance.keys())
    elif isinstance(p2, set):
        return p2.union(p1.occurance.keys())
    
    pn = PostingList()
    # pn.token = f'{p1.token} | {p2.token}'
    for pn1_keys in p1.occurrance.keys() :
        pn.addOccurrance(pn1_keys, p1.occurrance[pn1_keys])
    for pn2_keys in p2.occurrance.keys() :
        pn.addOccurrance(pn2_keys, p2.occurrance[pn2_keys])
    
    return pn

def inverse_posting(inverted_index,p):
    print(p)
    if isinstance(p, set) :
        print('Returning ')
        print(set(inverted_index.docs).difference(p))
        return set(inverted_index.docs).difference(p)
    else:
        print(set(inverted_index.docs).difference(set(p.occurrance.keys())))
        return set(inverted_index.docs).difference(set(p.occurrance.keys()))
    return inverted_index.docs.keys() - p.occurrance.keys()

In [10]:
    
def build_index():
    path_to_data = os.path.dirname(__file__) + '../../data/'
    print(os.path.dirname(__file__))
    print(path_to_data)
    vocab = set()
    doc_contents = []
    inverted_index = InvertedIndex()
    printable = set(string.printable) 
    # Printable characters are
    # 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
    # !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c


    ps = PorterStemmer()
    stop_words = set()
    with open(path_to_data+'Stopword-List.txt', 'r') as stop_word_file:
        lines = stop_word_file.readlines()
        for line in lines:
            stop_words.add(line.split('\n')[0])
        stop_words.remove('')
    print(stop_words)

    for file_number in range(0, 56):
        with open(path_to_data + f'Trump Speechs/speech_{file_number}.txt', 'r') as file1:
            lines = file1.readlines()
            print(f'File Number : speech_{file_number}.txt' )
            print(lines[0])
            position = {'doc':file_number,'row':0, 'col':0, 'token_no':0}

            for line_no,line in enumerate(lines):
                doc_set = set()
                # split words at . , whitespace ? ! : ;
                position['row'] = line_no 
                position['col'] = 0
                for word in re.split('[.\s,?!:;-]', line):
                    position['col'] += len(word) + 1
                    position['token_no'] += 1
                    # Case Folding
                    word = word.lower()
                    
                    # Filter non-ASCII characters
                    word = ''.join(filter(lambda x: x in printable, word))
                    
                    # Remove Punctuations
                    word = remove_punctuation(word)
                    
                    if re.match('\d+[A-Za-z]+',word):
                        word = re.split('\d+',word)[1]
                    if re.match('[A-Za-z]+\d+',word):
                        word = re.split('\d+',word)[0]
                    
                    if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
                        continue
                    if word in stop_words:
                        continue

                    word = ps.stem(word)
                        
                    vocab.add(word)
                    
                    doc_set.add(word)
                    
                    if word in inverted_index.index.keys():
                        
                        inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position)) 
                    else:
                        plist = PostingList()
                        inverted_index.index[word] = plist
                        inverted_index.index[word].addOccurrance(file_number, copy.deepcopy(position))
                        
            inverted_index.docs[file_number] = doc_set
            doc_contents.append(doc_set)
    ii = InvertedIndexModel()
    ii.status = True
    ii.data = inverted_index
    ii.save()
    return True


boperators = ['and', 'or']
uoperators = ['not']

# Token types
#
# EOF (end-of-file) token is used to indicate that
# there is no more input left for lexical analysis
LPAREN, RPAREN, EOF, TERM, AND, OR, NOT = (
    '(', ')', 'EOF', 'TERM', 'AND','OR', 'NOT'
)


class Token(object):
    def __init__(self, type, value):
        self.type = type
        self.value = value
        self.inverse = False
        self.row = []

    def __str__(self):
        """String representation of the class instance.

        Examples:
            Token(TERM, Hello)
            Token(AND, '&')
            Token(NOT, '!')
        """
        return 'Token({type}, {value})'.format(
            type=self.type,
            value=repr(self.value)
        )

    def __repr__(self):
        return self.__str__()


class Lexer(object):
    def __init__(self, text):
        # client string input, e.g. "hello | world & (why | are | you)"
        self.text = text
        # self.pos is an index into self.text
        self.pos = 0
        self.current_char = self.text[self.pos]
        

    def error(self):
        raise Exception('Invalid character')

    def advance(self):
        """Advance the `pos` pointer and set the `current_char` variable."""
        self.pos += 1
        if self.pos > len(self.text) - 1:
            self.current_char = None  # Indicates end of input
        else:
            self.current_char = self.text[self.pos]

    def skip_whitespace(self):
        while self.current_char is not None and self.current_char.isspace():
            self.advance()

    def integer(self):

        result = ''
        while self.current_char is not None and self.current_char.isdigit():
            result += self.current_char
            self.advance()
        return int(result)
    def word(self):
        result = ''
        # while self.current_char is not None and (self.current_char.isalpha() or self.current_char == '_'):
        while self.current_char is not None and (self.current_char in printable) and (self.current_char not in (' ', '|','&','!', '(', ')')):
            result += self.current_char
            self.advance()
        return str(result)

    def get_next_token(self):
        """Lexical analyzer (also known as scanner or tokenizer)

        This method is responsible for breaking a sentence
        apart into tokens. One token at a time.
        """
        while self.current_char is not None:

            if self.current_char.isspace():
                self.skip_whitespace()
                continue
                        
            if self.current_char == '&':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(AND, 'AND')
            
            if self.current_char == '|':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(OR, 'OR')
            
            if self.current_char == '!':
#                 print('gotB' + self.current_char)
                self.advance()
                return Token(NOT,'NOT')
            

            if self.current_char == '(':
                self.advance()
                return Token(LPAREN, '(')

            if self.current_char == ')':
                self.advance()
                return Token(RPAREN, ')')
            
            if self.current_char.isalpha():      
#                 print('Got token  ' + self.current_char)
                return Token(TERM, self.word())
            
            

            self.error()

        return Token(EOF, None)

class AST(object):
    pass


class BinOp(AST):
    def __init__(self, left, op, right):
        self.left = left
        self.token = self.op = op
        self.right = right
        self.inverse = False
        self.row = []
        self.value = ''
        
class Num(AST):
    def __init__(self, token):
        self.token = token
        self.value = token.value
        self.inverse = False
        self.row = []

class Parser(object):
    def __init__(self, lexer):
        self.lexer = lexer
        # set current token to the first token taken from the input
        self.current_token = self.lexer.get_next_token()

    def error(self):
        raise Exception('Invalid syntax')

    def eat(self, token_type):
        # compare the current token type with the passed token
        # type and if they match then "eat" the current token
        # and assign the next token to the self.current_token,
        # otherwise raise an exception.
        if self.current_token.type == token_type:
            self.current_token = self.lexer.get_next_token()
        else:
            self.error()

    def factor(self):
        """factor : INTEGER | LPAREN expr RPAREN"""
        token = self.current_token
        
        
        if token.type == TERM:
            self.eat(TERM)
            return Num(token)
        
        elif token.type == NOT:
            self.eat(NOT)
            node = self.expr()
            node.inverse = True
            return node
            
        
        
        elif token.type == LPAREN:
            self.eat(LPAREN)
            node = self.expr()
            self.eat(RPAREN)
            return node

    def term(self):
        
        node = self.factor()

        while self.current_token.type in (AND,):
            token = self.current_token
         
            if token.type == AND:
                self.eat(AND)
            
            node = BinOp(left=node, op=token, right=self.factor())

        return node

    def expr(self):
        node = self.term()

        while self.current_token.type in (OR,):
            token = self.current_token
            if token.type == OR:
                self.eat(OR)
            
            node = BinOp(left=node, op=token, right=self.term())

        return node

    def parse(self):
        return self.expr()


class NodeVisitor(object):
    def visit(self, node):
#         print('Checking Node Name')
        
        method_name = 'visit_' + type(node).__name__
        visitor = getattr(self, method_name, self.generic_visit)
        return visitor(node)

    def generic_visit(self, node):
        raise Exception('No visit_{} method'.format(type(node).__name__))


class Interpreter(NodeVisitor):
    def __init__(self, parser, index, ps):
        self.parser = parser
        self.index = index
        self.ps = ps

    def visit_BinOp(self, node):
#         print('Bin OP : ' )
#         print(node.token)
#         print(node.value)
#         print(node.row)
#         print(node.inverse)
        if node.op.type == AND:
#             print('Node => ')
#             print(node)
            left = self.visit(node.left)   
            right = self.visit(node.right)
            
#             term_index_left = vocab_list.index(ps.stem(left.value))
#             term_row_left = term_doc_matrix_np[term_index_left]
            
#             term_index_right = vocab_list.index(ps.stem(right.value))
#             term_row_right = term_doc_matrix_np[term_index_right]
            
            if left.inverse == True:
                left.value = '!' + str(left.value)
                term_row_left = self.index.index[self.ps.stem(left.row)]
                left.inverse = False
            
            if right.inverse == True:
                right.value = '!' + str(right.value)
                term_row_right = self.index.index[self.ps.stem(right.row)]
                right.inverse = False
            
            node.row = intersect_posting(left.row, right.row)
            if node.inverse == True:
                node.row = inverse_posting(self.index, node.row)
                node.inverse = False
            
            return node
        
        elif node.op.type == OR:
#             print('Node => ')
#             print(node)
            left = self.visit(node.left)   
            right = self.visit(node.right)
            
#             term_index_left = vocab_list.index(ps.stem(left.value))
#             term_row_left = term_doc_matrix_np[term_index_left]
            
#             term_index_right = vocab_list.index(ps.stem(right.value))
#             term_row_right = term_doc_matrix_np[term_index_right]
            
            if left.inverse == True:
                left.value = '!' + str(left.value)
                term_row_left = self.index.index([self.ps.stem(left.row)])
                left.inverse = False
            
            if right.inverse == True:
                right.value = '!' + str(right.value)
                term_row_right = self.index.index([self.ps.stem(right.row)])
                right.inverse = False
            

            node.row = union_posting(left.row, right.row)
            if node.inverse == True:
                node.row = inverse_posting(self.index, node.row)
                node.inverse = False
            
            return node
        

    def visit_Num(self, node):
#         print('Num  : ' )
#         print(node.token)
#         print(node.value)
#         print(node.row)
#         print(node.inverse)
        
        node.value = node.value.split('_')[0]
        if self.ps.stem(node.value) in self.index.index.keys():
            
            term_docs = self.index.index[self.ps.stem(node.value)]
            
        else:
            term_docs = {}
#             print('Term Row')
#             print(term_docs)
            
        node.row = term_docs
        if node.inverse == True:
            node.row = inverse_posting(self.index, node.row)
            node.inverse = False
        
        return node

    def interpret(self):
        tree = self.parser.parse()
#         print(tree)
        return self.visit(tree)

def get_boolean_query(query):
    text = str(query)
    text = text.replace(' and ','&')
    text = text.replace(' AND ','&')
    text = text.replace(' or ','|')
    text = text.replace(' OR ','|')
    text = text.replace('NOT', '!')
    text = text.replace('not ','!')
    
    print(text)
#     Django Specific Code
#     inverted_index_model_obj = InvertedIndexModel.objects.get()
#     inverted_index = inverted_index_model_obj.data
#     print('Inverted Index')
    
    ps = PorterStemmer()
    lexer = Lexer(text)
    parser = Parser(lexer)
    interpreter = Interpreter(parser, inverted_index, ps)
    result = interpreter.interpret()

#     print(result.value)
#     print(result.row)
    return result.row

def positional_intersect(p1, p2, k):
    
    ip = intersect_posting(p1, p2)
    
    lip = sorted(list(ip.occurrance))
    npl = PostingList()
    ans = []
    
    for doc in lip:
#         print(type(p1))
        positions1 = p1.occurrance[doc]
        positions2 = p2.occurrance[doc]
        index_p2 = 0
        index_p1 = 0
        for pos1 in positions1:
            for pos2 in positions2:
                if pos2['token_no'] -  pos1['token_no'] == k and pos2['token_no'] -  pos1['token_no'] > 0:
                    ans.append({'doc':doc, 'pos1':  pos1, 'pos2':pos2})
                    npl.addOccurrance(doc,pos1)
                    npl.addOccurrance(doc,pos2)
        
        
    return npl
        

def get_phrasal_query(query):
    text = str(query)

    try:
        q1, q2 = text.split(' ')
    except ValueError as e:
        raise ValueError('Invalid Phrasal Query Syntax')
    ps = PorterStemmer()
    q1 = ps.stem(q1)
    q2 = ps.stem(q2)
#     Django Specific Code
#     inverted_index_model_obj = InvertedIndexModel.objects.get()
#     inverted_index = inverted_index_model_obj.data
#     print('Inverted Index')
    result = [] 
    p1 = inverted_index.get_term_postings(q1)
    p2 = inverted_index.get_term_postings(q2)
    
    result = positional_intersect(p1, p2, 1)
    
    return result

def get_proximity_query(query):
    text = str(query)
    try:
        q1, q2, q3 = text.split(' ')
    except ValueError as e:
        raise ValueError('Invalid Proximity Query Syntax')
    ps = PorterStemmer()
    q1 = ps.stem(q1)
    q2 = ps.stem(q2)
    k = int(q3[1])+ 1
#     Django Specific Code
#     inverted_index_model_obj = InvertedIndexModel.objects.get()
#     inverted_index = inverted_index_model_obj.data
    
    print('Inverted Index')
    result = [] 
    
    p1 = inverted_index.get_term_postings(q1)
    p2 = inverted_index.get_term_postings(q2)
    print(p1)
    result = positional_intersect(p1, p2, k)
    

    return result

## Check Provided Queries:

In [11]:
fails = []

""" Boolean Queries """
q = 'running'
a = {'0', '1', '10', '11', '12', '16', '17', '18', '19', '2', '20', '21', '22', '24', '25', '26', '27', '28', '3', '30', '32', '33', '34', '35', '36', '37', '39', '4', '40', '41', '44', '45', '46', '47', '5', '50', '51', '52', '53', '6', '8', '9'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})

q = 'not hammer'
a = {'31', '28', '37', '30', '7', '10', '14', '1', '6', '41', '15', '11', '29', '26', '52', '13', '32', '44', '4', '8', '22', '38', '48', '0', '47', '2', '23', '9', '3', '5', '12', '55'}
r = get_boolean_query(q)
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'actions AND wanted'
a = {'37', '3', '19', '1', '9', '40', '51', '16', '15', '12', '31', '41', '39', '0', '53', '26', '29', '17', '24', '54', '7', '2', '5', '28', '42'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'policies AND western'
a ={'3', '2', '9'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'united OR plane'
a = {'31', '28', '50', '46', '37', '30', '54', '10', '18', '7', '1', '17', '41', '49', '6', '34', '36', '11', '45', '29', '26', '52', '13', '21', '24', '16', '25', '32', '33', '4', '44', '22', '8', '19', '40', '20', '38', '48', '0', '47', '27', '51', '43', '2', '35', '39', '9', '3', '5', '12', '55'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'amazing AND playing AND here'
a = {'51', '8', '0', '13', '39', '18', '19', '41', '20', '40', '16', '21', '50', '11', '27'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'higher AND signals AND enjoyed'
a = {'8'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'higher OR signals OR enjoyed'
a = {'8', '0', '39', '24', '41', '22', '44', '12', '23', '30', '31', '27', '20', '1', '40', '43', '19', '53', '7', '37', '2', '21'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'forgiveness OR developments OR praised'
a = {'3', '8', '18', '20', '32', '43', '9', '17', '36', '44', '52', '42', '37', '38', '16', '31', '5', '2'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'pakistan OR afghanistan OR aid'
a = {'29', '16', '4', '22', '37', '40', '42', '18', '1', '17', '41', '39', '9', '3'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'praised OR ( forgiveness AND developments )'
a = {'36', '42', '44', '2'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'officially OR (warriors AND cartoons)'
a = {'3', '0', '39', '18', '12', '48', '38', '5', '31', '10', '11', '47', '9', '17', '7', '37', '21', '25', '49'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'businessperson AND ( nationwide OR international )'
a = set({})
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'outdated AND ( personnel OR policies)'
a = {'2', '8', '5'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'outdated OR ( personnel AND policies )'
a = {'42', '8', '18', '5', '2', '11'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'biggest AND ( near OR box )'
a = {'18', '46', '54', '43', '4', '45', '50', '53', '47', '6', '51', '44'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'box AND ( united OR year )'
a = {'18', '46', '4', '45', '50', '9', '47', '23', '54', '44', '25'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'biggest AND ( plane OR wanted OR hour)'
a = {'50', '53', '46', '37', '30', '54', '42', '18', '7', '1', '49', '41', '6', '36', '45', '26', '52', '44', '16', '4', '8', '19', '40', '48', '0', '47', '51', '43', '2', '35', '39'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'NOT (united AND plane)'
a = {'31', '28', '50', '53', '46', '37', '54', '42', '7', '10', '14', '18', '6', '49', '41', '15', '11', '45', '13', '21', '44', '16', '4', '8', '22', '40', '20', '38', '48', '47', '51', '43', '23', '39', '9', '3', '5', '12', '55'}
r = get_boolean_query(q)
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'NOT (higher OR signals OR enjoyed)'
a = {'3', '52', '16', '36', '10', '14', '34', '9', '17', '42', '55', '50', '49', '4', '18', '13', '29', '48', '28', '5', '38', '26', '35', '45', '11', '54', '51', '6', '47', '32', '46', '15', '33', '25'}
r = get_boolean_query(q)
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})

""" phrasal queries """
q = 'Hillary Clinton' 
a = {'1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '14', '16', '17', '18', '19', '20', '21', '22', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})

""" positional queries"""
q = 'after years /1'
a = {'6', '7', '44'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'develop solutions /1'
a = {'5', '32'}
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})
    
q = 'keep out /2'
a = {'20', '24', '39', '40', '51'} 
r = get_boolean_query(q).occurrance.keys()
rs = set([str(x) for x in r])
if (len(a.difference(rs)) == 0):
    q
else:
    'Fail ' + q
fails.append({q:[a.difference(rs), rs.difference(a)]})

' Boolean Queries '

running


'running'

!hammer
total_cnt : 29 docs : [dict_keys([16, 17, 18, 19, 20, 21, 24, 25, 27, 33, 34, 35, 36, 39, 40, 42, 43, 45, 46, 49, 50, 51, 53, 54])]
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 22, 23, 26, 28, 29, 30, 31, 32, 37, 38, 41, 44, 47, 48, 52, 55}


'not hammer'

...

'after years /1'

develop solutions /1


'develop solutions /1'

keep out /2


'keep out /2'

In [12]:
# If Result is not Empty, Some Queries are not returning correct docs
list(filter(lambda k : 'FAILED : '+str(list(k.keys)[0]) if len(list(k.values())[0][0]) > 0 else None ,fails))

[]

In [13]:
# Test to check least frequent terms
# m = 100
# val = []
# for p in inverted_index.index:
#     print(p)
#     if len(p) < m:
#         m = len(p)
#         val = p

### Some test queries:

In [14]:
q = input('Enter Boolean Query: e.g outdated OR ( personnel AND policies ) : ')
get_boolean_query(q)
# Output contains posting list

Enter Boolean Query: e.g outdated OR ( personnel AND policies ) : outdated OR ( personnel AND policies )
outdated|( personnel&policies )


total_cnt : 7 docs : [dict_keys([2, 5, 8, 18, 11, 42])]

In [15]:
q = input('Enter phrasal Query: e.g Hillary Clinton')
get_phrasal_query(q)
# Output contains posting list

Enter phrasal Query: e.g Hillary ClintonHillary Clinton


total_cnt : 1116 docs : [dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54])]

In [16]:
q = input('Enter Proximity Query: e.g develop solutions /1 : ')
get_proximity_query(q)
# Output contains posting list

Enter Proximity Query: e.g develop solutions /1 : develop solutions /1
Inverted Index
total_cnt : 33 docs : [dict_keys([2, 3, 5, 8, 9, 16, 17, 18, 20, 31, 32, 37, 38])]


total_cnt : 4 docs : [dict_keys([5, 32])]

In [151]:
# set(inverted_index.docs).symmetric_difference(set(inverted_index.get_term_postings('candidaci').occurrance.keys()))
# set(inverted_index.get_term_postings('candidaci').occurrance.keys())

In [152]:
# set(inverted_index.get_term_postings('candidaci').occurrance.keys())

# References :

http://www.pyregex.com/
http://cs231n.github.io/python-numpy-tutorial/

https://www.online-utility.org/text/analyzer.jsp

https://stackoverflow.com/questions/2118261/parse-boolean-arithmetic-including-parentheses-with-regex

https://regex101.com/r/M8z3U4/1

https://iq.opengenus.org/porter-stemmer/

https://unnikked.ga/how-to-build-a-boolean-expression-evaluator-518e9e068a65

https://ruslanspivak.com/lsbasi-part7/

https://github.com/gintas/django-picklefield

In [6]:
from nltk.corpus import wordnet
synonyms = []
antonyms = []

for syn in wordnet.synsets("interest"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
             antonyms.append(l.antonyms()[0].name())

print(set(synonyms))
print(set(antonyms))

{'occupy', 'pursuit', 'sake', 'worry', 'concern', 'matter_to', 'interest_group', 'involvement', 'stake', 'interest', 'pastime', 'interestingness'}
{'uninterestingness', 'bore'}
